In [1]:
import pandas as pd

In [2]:
df1 = pd.read_parquet(path='../data/train.parquet', engine='pyarrow')
df2 = pd.read_parquet(path='../data/result.parquet', engine='pyarrow')

In [3]:
df1['label'] = 'train'
df2['label'] = 'predict'

# Concat
concat_df = pd.concat([df1 , df2])

In [4]:
concat_df.head()

,programevent_id,channel,market,turnover,weekday,period,date,time,participant_a,participant_b,label
0,33954710,Retail,standard,17795.0,Friday,Evening,2009-09-18,20:30:00,Schalke 04,VfL Wolfsburg,train
1,33954710,Retail,double-chance,4950.0,Friday,Evening,2009-09-18,20:30:00,Schalke 04,VfL Wolfsburg,train
2,33954710,Retail,next-point,4211.0,Friday,Evening,2009-09-18,20:30:00,Schalke 04,VfL Wolfsburg,train
3,33954710,Retail,points-more-less-than,2031.0,Friday,Evening,2009-09-18,20:30:00,Schalke 04,VfL Wolfsburg,train
4,33954710,Retail,points-more-less-rest,2587.0,Friday,Evening,2009-09-18,20:30:00,Schalke 04,VfL Wolfsburg,train


In [5]:
features = ['channel', 'market', 'weekday', 'period', 'participant_a', 'participant_b']

In [6]:
df = pd.get_dummies(data=concat_df, prefix='is_', columns=features).drop(labels=['programevent_id', 'date', 'time'], axis=1)

In [7]:
df = df.loc[:, ~df.columns.duplicated()]

In [8]:
df.head()

,turnover,label,is__Online,is__Retail,is__corner-kick-handicap,is__corner-kicks-more-less-rest,is__corner-kicks-more-less-rest-ht,is__corner-kicks-more-less-than,is__correct-score-live,is__custom-place-yes-no,...,is__SV Darmstadt 98,is__Schalke 04,is__VfB Stuttgart,is__VfL Wolfsburg,is__Werder Bremen,is__B. Leverkusen,is__B. M'gladbach,is__Darmstadt 98,is__FC Augsburg,is__Wolfsburg
0,17795.0,train,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,4950.0,train,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,4211.0,train,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,2031.0,train,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,2587.0,train,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [ ]:
pd.options.display.max_seq_items = 2000

In [10]:
import numpy as np

In [11]:
features = np.delete(arr=df.columns.values, obj=np.where((df.columns.values == 'turnover') | (df.columns.values == 'label')))

In [12]:
# Split your data
df1 = df[df['label'] == 'train']
df2 = df[df['label'] == 'predict']

In [13]:
# Drop your labels
df1 = df1.drop(labels='label', axis=1)
df2 = df2.drop(labels='label', axis=1)

In [14]:
df2.to_parquet(fname='../data/predict.parquet', engine='pyarrow')

In [15]:
from sklearn.tree import DecisionTreeRegressor

In [16]:
# values of features
X = np.array(df1[features].values)

# target values
y = list(df1['turnover'])

In [17]:
mdl = DecisionTreeRegressor().fit(X, y)
print(mdl)

# make predictions
pred = mdl.predict(df1[features])

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')


In [18]:
import pickle
s = pickle.dump(mdl, open("../model/mdl.obj", "wb"))